참고문헌: PyTorch로 시작하는 딥 러닝 입문 (wikidocs), 파이썬 딥러닝 파이토치 (이경택,방성수, 안상준 지음), 정보문화사


# 선형 회귀(Linear Regression)
---

이번 챕터에서는 선형 회귀 이론에 대해서 이해하고, 파이토치(PyTorch)를 이용하여 선형 회귀 모델을 만들어보겠습니다.

<img src = "https://raw.githubusercontent.com/Hyun-chul/KAIA2022/main/Regression.png">

*  데이터에 대한 이해(Data Definition)
*  학습할 데이터에 대해서 알아봅니다.


*  가설(Hypothesis) 수립
*  가설을 수립하는 방법에 대해서 알아봅니다.


*  손실 계산하기(Compute loss)
*  학습 데이터를 이용해서 연속적으로 모델을 개선시키는데 이 때 손실(loss)를 이용합니다.


*  경사 하강법(Gradient Descent)
*  학습을 위한 핵심 알고리즘인 경사 하강법(Gradient Descent)에 대해서 이해합니다.


- 데이터에 대한 이해(Data Definition)

    이번 챕터에서 선형 회귀를 위해 사용할 예제는 공부한 시간과 점수에 대한 상관관계입니다.

-  훈련 데이터셋과 테스트 데이터셋

    어떤 학생이 1시간 공부를 했더니 2점, 다른 학생이 2시간 공부를 했더니 4점, 또 다른 학생이 3시간을 공부했더니 6점을 맞았습니다. 그렇다면, 내가 4시간을 공부한다면 몇 점을 맞을 수 있을까요?



1. 기본 셋팅

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# 현재 실습하고 있는 파이썬 코드를 재실행해도 다음에도 같은 결과가 나오도록 랜덤 시드(random seed)를 줍니다.
torch.manual_seed(1)

실습을 위한 기본적인 셋팅이 끝났습니다. 이제 훈련 데이터인 x_train과 y_train을 선언합니다.

2. 변수 선언


In [ ]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

x_train과 x_train의 크기(shape)를 출력해보겠습니다.

In [ ]:
print(x_train)
print(x_train.shape)

x_train의 값이 출력되고, x_train의 크기가 (3 × 1)임을 알 수 있습니다. 

y_train과 y_train의 크기(shape)를 출력해보겠습니다.

In [ ]:
print(y_train)
print(y_train.shape)

y_train의 값이 출력되고, y_train의 크기가 (3 × 1)임을 알 수 있습니다.

3. 가중치와 편향의 초기화

선형 회귀란 학습 데이터와 가장 잘 맞는 하나의 직선을 찾는 일입니다.

그리고 가장 잘 맞는 직선을 정의하는 것은 바로 *W* 와 *b* 입니다. 

선형 회귀의 목표는 가장 잘 맞는 직선을 정의하는 *W* 와 *b* 값을 찾는 것입니다.

우선 가중치 *W* 를 0으로 초기화하고, 이 값을 출력해보겠습니다.



In [ ]:
# 가중치 W를 0으로 초기화하고 학습을 통해 값이 변경되는 변수임을 명시함.
W = torch.zeros(1, requires_grad=True) 
# 가중치 W를 출력
print(W) 

가중치 *W* 가 0으로 초기화되어있으므로 0이 출력된 것을 확인할 수 있습니다. 위에서 requires_grad=True가 인자로 주어진 것을 확인할 수 있습니다. 이는 이 변수는 학습을 통해 계속 값이 변경되는 변수임을 의미합니다.

마찬가지로 편향 *b*도 0으로 초기화하고, 학습을 통해 값이 변경되는 변수임을 명시합니다.

In [ ]:
b = torch.zeros(1, requires_grad=True)
print(b)

현재 가중치 $W$와 $b$ 둘 다 0이므로 현 직선의 방정식은 다음과 같습니다.

$y = Wx + b$

지금 상태에선 $x$에 어떤 값이 들어가도 가설은 0을 예측하게 됩니다. 즉, 아직 적절한 $W$와 $b$의 값이 아닙니다.

4. 가설 세우기


파이토치 코드 상으로 직선의 방정식에 해당되는 가설을 선언합니다.

$H(x) = Wx + b$

In [ ]:
hypothesis = x_train * W + b
print(hypothesis)

5. 비용 함수 선언하기


파이토치 코드 상으로 선형 회귀의 비용 함수에 해당되는 평균 제곱 오차를 선언합니다.


$cost(W, b) = \frac{1}{n} \sum_{i=1}^{n} \left[y^{(i)} - H(x^{(i)})\right]^2$

In [ ]:
# 앞서 배운 torch.mean으로 평균을 구한다.
cost = torch.mean((hypothesis - y_train) ** 2) 
print(cost)

6. 경사 하강법 (Gradient descent) 구현하기

이제 경사 하강법을 구현합니다. 아래의 'stochastic gradient descent(SGD)'는 경사 하강법의 일종입니다. lr은 학습률(learning rate, $\alpha$)를 의미합니다.
학습 대상인 W와 b가 SGD의 입력이 됩니다.

<figure>
<img src = "https://raw.githubusercontent.com/Hyun-chul/KAIA2022/main/GD.png"/, height = 200, width = 300>
</figure>


$H(x) = Wx + b$


$cost(W, b) = \frac{1}{n} \sum_{i=1}^{n} \left[y^{(i)} - H(x^{(i)})\right]^2$

기울기 = $\frac{\partial cost(W)}{\partial W}$	

$ W := W - α\frac{\partial cost(W)}{\partial W} $

> optimizer = optim.SGD([W, b], lr=0.01)

optimizer.zero_grad()를 실행하므로서 미분을 통해 얻은 기울기를 0으로 초기화합니다. 기울기를 초기화해야만 새로운 가중치 편향에 대해서 새로운 기울기를 구할 수 있습니다. 

그 다음 cost.backward() 함수를 호출하면 가중치 W와 편향 b에 대한 기울기가 계산됩니다. 그 다음 경사 하강법 최적화 함수 opimizer의 .step() 함수를 호출하여 인수로 들어갔던 W와 b에서 리턴되는 변수들의 기울기에 학습률(learining rate) 0.01을 곱하여 빼줌으로서 업데이트합니다.


#
gradient를 0으로 초기화

> optimizer.zero_grad()

# 
비용 함수를 미분하여 gradient 계산
> cost.backward() 

# 
W와 b를 업데이트

> optimizer.step() 

7. 전체 코드

In [ ]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 1999 # 원하는만큼 경사 하강법을 반복
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x_train * W + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

In [ ]:
import matplotlib.pyplot as plt

overlapping = 0.150

print(hypothesis)
# print(hypothesis.detach())

# prect_y = hypothesis.detach() #  기존 Tensor에서 gradient 전파가 안되는 텐서 생성
prect_y = hypothesis #  기존 Tensor에서 gradient 전파가 안되는 텐서 생성

line1 = plt.plot(prect_y.detach().numpy(), c='red', alpha=overlapping, lw=5)
line2 = plt.plot(y_train, c='green', alpha=overlapping, lw=5)

plt.show()

8. 전체 코드 (실시간 결과 보기)

In [ ]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])
# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 1999 # 원하는만큼 경사 하강법을 반복
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x_train * W + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))
        prect_y = hypothesis.detach() # 기존 Tensor에서 gradient 전파가 안되는 텐서 생성

        line1 = plt.plot(prect_y, c='red', alpha=overlapping,lw=5)
        line2 = plt.plot(y_train, c='green', alpha=overlapping,lw=5)

        plt.show()

9. 전체 코드 (에러값 저장하기)

In [ ]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])
# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.0001)

nb_epochs = 1999 # 원하는만큼 경사 하강법을 반복

# cost 값을 저장할 수 있는 variable 선언
cost_vals = torch.zeros(nb_epochs+1)
print(cost_vals)

for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x_train * W + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    cost_vals[epoch] = cost.item()
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))


In [ ]:
print(cost_vals)

line1 = plt.plot(cost_vals, c='red',lw=5)
plt.show()